In [29]:
from pykrx import stock
import yfinance as yf
from datetime import datetime, time, timedelta

In [11]:
start_date = '20220720'
end_date = '20220810'
stock_code = '005930'

In [16]:
def to_yfinance_time_format(YYYYMMDD:str):
    '''
    YYYYMMDD 형식으로 통일한 본 시스템의 input을 YYYY-MM-DD로 변환
    '''
    year = YYYYMMDD[:4]
    month = YYYYMMDD[4:6]
    date = YYYYMMDD[6:]
    output_format = f'{year}-{month}-{date}'
    return output_format

In [30]:
def get_real_time_OHLCV(stock_code):
    '''
    yfinance에서 현재 시간 기준으로 호출된 날짜의 개장 후 5분봉 데이터를 실시간으로 조회 후 반환
    '''
    try:
        # 종목 코드를 yfinance 형식에 맞게 변환
        stock_code = f'{stock_code}.KS'

        # 현재 시간
        end_time = datetime.now()

        # 당일 개장 시각 (오전 9시)
        market_open_time = datetime.combine(end_time.date(), time(9, 0))

        # 현재 시간이 개장 전일 경우 예외 처리
        if end_time < market_open_time:
            raise ValueError("시장이 아직 개장하지 않았습니다.")

        # 데이터 다운로드
        data = yf.download(stock_code, start=market_open_time, end=end_time, interval='5m')

        return data
    
    except Exception as e:
        print(f"yfinance 실시간 데이터 Fetch 실패: {e}")
        return None

In [36]:
def get_daily_OHLCV(stock_code:str, start_date:str, end_date:str):
    '''
    지정된 일자 간격에 대해 OHLCV 정보를 1시간봉 기준으로 조회 후 df형태로 반환
    '''
    try:
        stock_code = f'{stock_code}.KS'
        start_date = to_yfinance_time_format(start_date)
        end_date = to_yfinance_time_format(end_date)
        data = yf.download(stock_code, start=start_date, end=end_date, interval='1h')
        return data
    except Exception as e:
        print(f"yfinance 일자간 가격정보 Fetch 실패: {e}")
        return None

In [37]:
price = get_daily_OHLCV('005930', '20240801', '20240810')

price

[*********************100%***********************]  1 of 1 completed

,Open,High,Low,Close,Adj Close,Volume
Datetime,,,,,,
2024-08-01 09:00:00+09:00,86000.0,86100.0,83500.0,84000.0,84000.0,0
2024-08-01 10:00:00+09:00,83900.0,84200.0,83400.0,83700.0,83700.0,2357162
2024-08-01 11:00:00+09:00,83800.0,84200.0,83600.0,83900.0,83900.0,1308424
2024-08-01 12:00:00+09:00,84000.0,84300.0,83800.0,84200.0,84200.0,1257391
2024-08-01 13:00:00+09:00,84100.0,84200.0,83800.0,83900.0,83900.0,1304206
2024-08-01 14:00:00+09:00,83800.0,83900.0,83400.0,83600.0,83600.0,1728867
2024-08-02 09:00:00+09:00,81000.0,81400.0,80700.0,81000.0,81000.0,7005416
2024-08-02 10:00:00+09:00,81000.0,81000.0,80500.0,80600.0,80600.0,2836755
2024-08-02 11:00:00+09:00,80500.0,80600.0,80000.0,80300.0,80300.0,3478811
